In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import numpy as np
import math


In [ ]:
pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQFXqrZBnbYQjmUMFwlQ7WCnZNtsBCW8DkiJuNBFCJnIRRuiUG_9bdnFrjgKF-uNaOqhzid8B87v1ZE/pub?gid=501617339&single=true&output=csv')
data

,symbol,sector
0,AAI,agro
1,APURE,agro
2,ASIAN,agro
3,BR,agro
4,BRR,agro
...,...,...
691,WHAIR,propcon
692,WHART,propcon
693,WIIK,propcon
694,WIN,propcon


In [ ]:
data['symbol'] = data['symbol']+'.BK'
data

,symbol,sector
0,AAI.BK,agro
1,APURE.BK,agro
2,ASIAN.BK,agro
3,BR.BK,agro
4,BRR.BK,agro
...,...,...
691,WHAIR.BK,propcon
692,WHART.BK,propcon
693,WIIK.BK,propcon
694,WIN.BK,propcon


In [ ]:
agro_col = data[data['sector'] == 'agro']
tech_col = data[data['sector'] == 'tech']
resourc_col = data[data['sector'] == 'resourc']
fincial_col = data[data['sector'] == 'fincial']
service_col = data[data['sector'] == 'service']
indus_col = data[data['sector'] == 'indus']
consump_col = data[data['sector'] == 'consump']
propcon_col = data[data['sector'] == 'propcon']
agro_col

,symbol,sector
0,AAI.BK,agro
1,APURE.BK,agro
2,ASIAN.BK,agro
3,BR.BK,agro
4,BRR.BK,agro
...,...,...
62,UPOIC.BK,agro
63,UVAN.BK,agro
64,VPO.BK,agro
65,W.BK,agro


In [ ]:
data_bk = data['symbol']
df_bk = pd.DataFrame(data_bk)
df_bk

,symbol
0,AAI.BK
1,APURE.BK
2,ASIAN.BK
3,BR.BK
4,BRR.BK
...,...
691,WHAIR.BK
692,WHART.BK
693,WIIK.BK
694,WIN.BK


In [ ]:
sectors = ['agro', 'tech', 'resourc', 'fincial', 'service', 'indus', 'consump', 'propcon']
tickers_by_sector = [agro_col, tech_col, resourc_col, fincial_col, service_col, indus_col, consump_col, propcon_col]
weights_by_sector = {
    'agro': 0.25,
    'tech': 0.1,
    'resourc': 0.3,
    'fincial': 0.05,
    'service': 0.1,
    'indus': 0.1,
    'consump': 0.05,
    'propcon': 0.05
}

In [ ]:
final_weights_by_sector = {}

for sector, tickers in zip(sectors, tickers_by_sector):
    try:
        # Download stock data
        stock_data = yf.download(tickers['symbol'].tolist(), start=pd.Timestamp.today() - pd.DateOffset(years=1), end=pd.Timestamp.today())
        df = stock_data['Adj Close']

        # Calculate expected returns and covariance matrix
        mu = expected_returns.mean_historical_return(df)
        Sigma = risk_models.sample_cov(df)

        # Optimize portfolio
        ef = EfficientFrontier(mu, Sigma)
        weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights()

        # Store optimized weights
        final_weights_by_sector[sector] = cleaned_weights

        # Display results
        print(f"Optimal weights for {sector} sector:")
        print(cleaned_weights)
        ef.portfolio_performance(verbose=True)

    except Exception as e:
        print(f"Error occurred: {e}")

# Calculate total weights sum for each sector
for sector, weights in final_weights_by_sector.items():
    total_weights_sum = sum(weights.values())
    final_weights_by_sector[sector] = {ticker: weight / total_weights_sum for ticker, weight in weights.items()}
    print(f"\nFinal weights for {sector} sector:", final_weights_by_sector[sector])

# # Calculate final weights for the overall portfolio case equal all sector
# all_weights = {ticker: 0 for ticker in data_bk.tolist()}
# for sector, weights in final_weights_by_sector.items():
#     for ticker, weight in weights.items():
#         if ticker in all_weights:
#             all_weights[ticker] += weight
#         else:
#             print(f"Ticker {ticker} not found in all_weights")

# total_weights_sum = sum(all_weights.values())
# final_weights = {ticker: weight / total_weights_sum for ticker, weight in all_weights.items()}
# print("\nFinal weights for the overall portfolio:", final_weights)




[*********************100%%**********************]  67 of 67 completed
[**                     5%%                      ]  2 of 44 completed

Optimal weights for agro sector:
OrderedDict([('AAI.BK', 0.0), ('APURE.BK', 0.0), ('ASIAN.BK', 0.0), ('BR.BK', 0.0), ('BRR.BK', 0.0), ('BTG.BK', 0.0), ('CBG.BK', 0.0), ('CFRESH.BK', 0.0), ('CH.BK', 0.0), ('CHOTI.BK', 0.0), ('CM.BK', 0.0), ('COCOCO.BK', 0.04856), ('CPF.BK', 0.0), ('CPI.BK', 0.0), ('EE.BK', 0.0), ('F&D.BK', 0.0), ('GFPT.BK', 0.0), ('GLOCON.BK', 0.0), ('HTC.BK', 0.0), ('ICHI.BK', 0.0), ('ITC.BK', 0.0), ('JDF.BK', 0.0), ('KBS.BK', 0.0), ('KCG.BK', 0.0491), ('KSL.BK', 0.0), ('KTIS.BK', 0.0), ('LEE.BK', 0.0), ('LST.BK', 0.0), ('M.BK', 0.0), ('MALEE.BK', 0.18562), ('NER.BK', 0.0), ('NRF.BK', 0.0), ('NSL.BK', 0.0), ('OSP.BK', 0.0), ('PB.BK', 0.0), ('PLUS.BK', 0.0), ('PM.BK', 0.0), ('PPPM.BK', 0.0), ('PQS.BK', 0.0), ('PRG.BK', 0.0), ('RBF.BK', 0.0), ('SAPPE.BK', 0.12104), ('SAUCE.BK', 0.27318), ('SNNP.BK', 0.0), ('SNP.BK', 0.0), ('SORKON.BK', 0.0), ('SSC.BK', 0.0), ('SSF.BK', 0.0), ('SST.BK', 0.0), ('STA.BK', 0.0), ('SUN.BK', 0.0), ('TC.BK', 0.0), ('TEGH.BK', 0.0), ('TFG.BK', 0

[*********************100%%**********************]  44 of 44 completed
[*                      3%%                      ]  2 of 69 completed

Optimal weights for tech sector:
OrderedDict([('3BBIF.BK', 0.0), ('ADVANC.BK', 0.0), ('AIT.BK', 0.0), ('ALT.BK', 0.0), ('AMR.BK', 0.0), ('BLISS.BK', 0.0), ('CCET.BK', 0.0134), ('DELTA.BK', 0.0), ('DIF.BK', 0.0), ('DITTO.BK', 0.0), ('DTCENT.BK', 0.0), ('FORTH.BK', 0.0), ('GABLE.BK', 0.0), ('HANA.BK', 0.0), ('HUMAN.BK', 0.0), ('ICN.BK', 0.0), ('ILINK.BK', 0.0), ('INET.BK', 0.0), ('INSET.BK', 0.0), ('INTUCH.BK', 0.0), ('ITEL.BK', 0.0), ('JAS.BK', 0.0), ('JMART.BK', 0.0), ('JTS.BK', 0.0958), ('KCE.BK', 0.0), ('METCO.BK', 0.0), ('MFEC.BK', 0.0), ('MSC.BK', 0.0), ('PT.BK', 0.52597), ('SAMART.BK', 0.22234), ('SAMTEL.BK', 0.0), ('SDC.BK', 0.0), ('SIS.BK', 0.0), ('SKY.BK', 0.0), ('SMT.BK', 0.0), ('SVI.BK', 0.0), ('SVOA.BK', 0.0), ('SYMC.BK', 0.14249), ('SYNEX.BK', 0.0), ('TEAM.BK', 0.0), ('THCOM.BK', 0.0), ('TKC.BK', 0.0), ('TRUE.BK', 0.0), ('TWZ.BK', 0.0)])
Expected annual return: 36.5%
Annual volatility: 20.9%
Sharpe Ratio: 1.65


[*********************100%%**********************]  69 of 69 completed
[                       0%%                      ]

...

Optimal weights for indus sector:
OrderedDict([('2S.BK', 0.0), ('3K-BAT.BK', 0.0), ('ACG.BK', 0.0), ('AH.BK', 0.0), ('AJ.BK', 0.0), ('ALLA.BK', 0.28166), ('ALUCON.BK', 0.0), ('AMC.BK', 0.14043), ('ASEFA.BK', 0.0), ('BCT.BK', 0.0), ('BGC.BK', 0.0), ('BSBM.BK', 0.0), ('CEN.BK', 0.0), ('CITY.BK', 0.0), ('CMAN.BK', 0.0), ('CPT.BK', 0.0), ('CRANE.BK', 0.0), ('CSC.BK', 0.0), ('CSP.BK', 0.0), ('CTW.BK', 0.0), ('CWT.BK', 0.0), ('EASON.BK', 0.0), ('FMT.BK', 0.08117), ('GC.BK', 0.0), ('GGC.BK', 0.0), ('GIFT.BK', 0.14601), ('GJS.BK', 0.0), ('GSTEEL.BK', 0.0), ('GYT.BK', 0.0), ('HFT.BK', 0.0), ('HTECH.BK', 0.0), ('IHL.BK', 0.0), ('INGRS.BK', 0.0), ('INOX.BK', 0.0), ('IRC.BK', 0.0), ('IVL.BK', 0.0), ('KKC.BK', 0.0), ('LHK.BK', 0.0), ('MCS.BK', 0.0), ('MGC.BK', 0.0), ('MILL.BK', 0.0), ('NEP.BK', 0.0), ('NEX.BK', 0.0), ('NFC.BK', 0.0), ('PAP.BK', 0.0), ('PATO.BK', 0.0), ('PCSGH.BK', 0.0), ('PERM.BK', 0.0), ('PJW.BK', 0.0), ('PK.BK', 0.0), ('PMTA.BK', 0.0), ('POLY.BK', 0.0), ('PSP.BK', 0.0), ('PTL.BK'

[*********************100%%**********************]  45 of 45 completed
[                       0%%                      ]

Optimal weights for consump sector:
OrderedDict([('AFC.BK', 0.0), ('AJA.BK', 0.01145), ('APCO.BK', 0.0), ('AURA.BK', 0.0), ('BIZ.BK', 0.0), ('BLC.BK', 0.0), ('BTNC.BK', 0.0), ('CPH.BK', 0.0), ('CPL.BK', 0.0), ('DDD.BK', 0.0), ('DTCI.BK', 0.0), ('FANCY.BK', 0.0), ('FTI.BK', 0.0), ('JCT.BK', 0.0), ('KISS.BK', 0.0), ('KYE.BK', 0.23537), ('L&E.BK', 0.0), ('MODERN.BK', 0.0), ('NAM.BK', 0.0), ('NC.BK', 0.1032), ('NV.BK', 0.0), ('OCC.BK', 0.01066), ('OGC.BK', 0.0), ('PAF.BK', 0.0), ('PDJ.BK', 0.0), ('PG.BK', 0.0), ('ROCK.BK', 0.06099), ('S&J.BK', 0.22121), ('SABINA.BK', 0.0), ('SAWANG.BK', 0.00372), ('SBNEXT.BK', 0.0), ('SIAM.BK', 0.0), ('STGT.BK', 0.0), ('STHAI.BK', 0.0), ('SUC.BK', 0.19892), ('TCMC.BK', 0.0), ('TNL.BK', 0.0), ('TNR.BK', 0.0), ('TOG.BK', 0.15448), ('TR.BK', 0.0), ('TTI.BK', 0.0), ('TTT.BK', 0.0), ('UPF.BK', 0.0), ('WACOAL.BK', 0.0), ('WFX.BK', 0.0)])
Expected annual return: 41.3%
Annual volatility: 19.0%
Sharpe Ratio: 2.07


[*********************100%%**********************]  170 of 170 completed
/usr/local/lib/python3.10/dist-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


Optimal weights for propcon sector:
OrderedDict([('A.BK', 0.0), ('AIMCG.BK', 0.0), ('AIMIRT.BK', 0.0), ('AKS.BK', 0.0), ('ALLY.BK', 0.0), ('AMATA.BK', 0.0), ('AMATAR.BK', 0.0), ('AMATAV.BK', 0.0), ('ANAN.BK', 0.0), ('AP.BK', 0.0), ('APCS.BK', 0.0), ('APEX.BK', 0.0), ('ASW.BK', 0.0), ('AWC.BK', 0.0), ('B-WORK.BK', 0.0), ('BAREIT.BK', 0.0), ('BJCHI.BK', 0.0), ('BKD.BK', 0.0), ('BKKCP.BK', 0.0), ('BLAND.BK', 0.0), ('BOFFICE.BK', 0.0), ('BRI.BK', 0.0), ('BROCK.BK', 0.0), ('CCP.BK', 0.0), ('CGD.BK', 0.0), ('CI.BK', 0.0), ('CIVIL.BK', 0.0), ('CK.BK', 0.0), ('CMC.BK', 0.0), ('CNT.BK', 0.0), ('CPN.BK', 0.0), ('CPNCG.BK', 0.0), ('CPNREIT.BK', 0.0), ('CPTGF.BK', 0.0), ('CTARAF.BK', 0.0), ('DCC.BK', 0.0), ('DCON.BK', 0.0), ('DREIT.BK', 0.0), ('DRT.BK', 0.0), ('EMC.BK', 0.0), ('EPG.BK', 0.0), ('ERWPF.BK', 0.30937), ('ESTAR.BK', 0.0), ('EVER.BK', 0.0), ('FPT.BK', 0.0), ('FTREIT.BK', 0.0), ('FUTUREPF.BK', 0.0), ('GAHREIT.BK', 0.0), ('GEL.BK', 0.0), ('GLAND.BK', 0.0), ('GROREIT.BK', 0.0), ('GVREIT.BK

In [ ]:
# Calculate final weights for the overall portfolio
all_weights = {ticker: 0 for ticker in data_bk.tolist()}
for sector, sector_weight in weights_by_sector.items():
    for ticker, weight in final_weights_by_sector[sector].items():
        if ticker in all_weights:
            all_weights[ticker] += weight * sector_weight
        else:
            print(f"Ticker {ticker} not found in all_weights")

total_weights_sum = sum(all_weights.values())
final_weights = {ticker: weight / total_weights_sum for ticker, weight in all_weights.items()}
print("\nFinal weights for the overall portfolio:", final_weights)



Final weights for the overall portfolio: {'AAI.BK': 0.0, 'APURE.BK': 0.0, 'ASIAN.BK': 0.0, 'BR.BK': 0.0, 'BRR.BK': 0.0, 'BTG.BK': 0.0, 'CBG.BK': 0.0, 'CFRESH.BK': 0.0, 'CH.BK': 0.0, 'CHOTI.BK': 0.0, 'CM.BK': 0.0, 'COCOCO.BK': 0.012140121401214014, 'CPF.BK': 0.0, 'CPI.BK': 0.0, 'EE.BK': 0.0, 'F&D.BK': 0.0, 'GFPT.BK': 0.0, 'GLOCON.BK': 0.0, 'HTC.BK': 0.0, 'ICHI.BK': 0.0, 'ITC.BK': 0.0, 'JDF.BK': 0.0, 'KBS.BK': 0.0, 'KCG.BK': 0.012275122751227514, 'KSL.BK': 0.0, 'KTIS.BK': 0.0, 'LEE.BK': 0.0, 'LST.BK': 0.0, 'M.BK': 0.0, 'MALEE.BK': 0.046405464054640556, 'NER.BK': 0.0, 'NRF.BK': 0.0, 'NSL.BK': 0.0, 'OSP.BK': 0.0, 'PB.BK': 0.0, 'PLUS.BK': 0.0, 'PM.BK': 0.0, 'PPPM.BK': 0.0, 'PQS.BK': 0.0, 'PRG.BK': 0.0, 'RBF.BK': 0.0, 'SAPPE.BK': 0.030260302603026035, 'SAUCE.BK': 0.06829568295682958, 'SNNP.BK': 0.0, 'SNP.BK': 0.0, 'SORKON.BK': 0.0, 'SSC.BK': 0.0, 'SSF.BK': 0.0, 'SST.BK': 0.0, 'STA.BK': 0.0, 'SUN.BK': 0.0, 'TC.BK': 0.0, 'TEGH.BK': 0.0, 'TFG.BK': 0.0, 'TFM.BK': 0.0, 'TFMAMA.BK': 0.04850548505

In [ ]:
final_weights_list = list(final_weights.items())
df_final_weights_list = pd.DataFrame(final_weights_list, columns=['symbol', 'weight'])
df_final_weights_list

,symbol,weight
0,AAI.BK,0.0
1,APURE.BK,0.0
2,ASIAN.BK,0.0
3,BR.BK,0.0
4,BRR.BK,0.0
...,...,...
691,WHAIR.BK,0.0
692,WHART.BK,0.0
693,WIIK.BK,0.0
694,WIN.BK,0.0


In [ ]:
df_final_weights_list_sorted = df_final_weights_list.sort_values(by='weight', ascending=False)
df_final_weights_list_sorted

,symbol,weight
121,BCP.BK,0.199968
42,SAUCE.BK,0.068296
172,TGE.BK,0.064122
95,PT.BK,0.052597
55,TFMAMA.BK,0.048505
...,...,...
243,TLI.BK,0.000000
244,TNITY.BK,0.000000
245,TQM.BK,0.000000
246,TSI.BK,0.000000


In [ ]:
df_final_weights_list['weight'].sum()

1.0

In [ ]:
df_final_weights_list_sorted.to_csv('vtwi;.csv')